## Dekorátor

Návrhový vzor, detailně popsaný např. na webu [refactoring.guru](https://refactoring.guru/design-patterns/decorator).

![UML diagram dekorátoru, refactoring.guru](https://refactoring.guru/images/patterns/diagrams/decorator/structure.png)

Zjednodušeně se dá říct, že dekorátor umožňuje opakovatelným způsobem rozšiřovat funkcionalitu existujícího kódu tak, že jej obalí dalším kódem (je to vlastně takový wrapper).

V pythonu mají dekorátory zvláštní postavení - máme k dispozici zjednodušující syntaxi pro jejich použití (jakýsi [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar)).

Ukažme si to na jednoduché funkci.

In [ ]:
def add(x, y):
    return x + y

add(1, 2)

Zkusme tuto funkci o něco rozšířit, např. o oznámení, že byla zavolaná, ale nesahejme na její definici. Jedna možnost, jak to udělat, je napsat wrapper - novou funkci, která tu původní obalí.

In [ ]:
def wrapper(x, y):
    print("calling funcion add")
    return add(x, y)

wrapper(1, 2)

Napišme si trochu obecnější wrapper: napišme funkci, která dostane obalovanou funkci na vstupu, a vrátí obalený výsledek. Jméno funkce najdeme pod atributem `__name__`.

In [ ]:
def better_wrapper(func):
    def wrapper(x, y):
        print(f"calling function {func.__name__}")
        return func(x, y)
    return wrapper

wrapped_add = better_wrapper(add)
wrapped_add(1, 2)

Teď můžeme obalit i jinou funkci.

In [ ]:
def multiply(x, y):
    return x * y

wrapped_multiply = better_wrapper(multiply)
wrapped_multiply(1, 2)

Jedinou slabinou je, že náš `better_wrapper` stále předpokládá, že obalovaná funkce přijímá dva argumenty. Můžeme napsat obecný wrapper. Vzhledem k tomu, co dělá, ho pojmenujme `log`

In [ ]:
def log(func):
    def wrapper(*args, **kwargs):
        print(f"calling function {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

Takový wrapper už umí obalit úplně libovolnou funkci.

In [ ]:
def some_function(arg1, **kwargs):
    print(arg1, kwargs.keys())
    
logged_some_function = log(some_function)

logged_some_function(True, x=3)

Funkce, která vrácí obalenou funkci, je vlastně dekorátorem (rozšiřuje funkcionalitu existujícího objektu). Pro komfort je možné v pythonu dekorovat funkce již při definici - nemusíme zavádět nová jména pro dekorované varianty. V pythonu k tomu slouží následující syntaxe

In [ ]:
@log
def another_function():
    print("this function does not actually do anything")
    
another_function()

Můžeme si dovolit ještě jednu úroveň abstrakce. Chování dekorátoru může být závislé na nějakém další parametru. Potřebujeme tedy napsat funkci, která nám vrátí dekorátor. Ale dekorátor je funkce, která vrací funkci. Takže napíšeme funkci, která vrací funkci, která vrací funkci.

Přidejme k našemu dekorátoru možnost logování vypnout.

In [ ]:
def log(do_log):
    def dec(func):
        def wrapper(*args, **kwargs):
            if do_log:
                print(f"calling function {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return dec

@log(True)
def add(x, y):
    return x + y

@log(False)
def multiply(x, y):
    return x * y

add(1, 2)
multiply(1, 2)

Tedy funkce `log` vyrábí různé dekorátory v závislosti na tom, jaký argument jí předáme. Pokud je `do_log==True`, pak dostaneme dekorátor, který k dekorované funkci přidá `print`. Pokud `do_log==False`, dostaneme triviální dekorátor, který nic nedělá.

Takový argument pak může sídlit v nějaké globální proměnné reprezentující nastavení (to je samo o sobě trochu nevhodné, ale o tom jindy):

In [ ]:
enable_logging = True


@log(enable_logging)
def add(x, y):
    return x + y

@log(enable_logging)
def multiply(x, y):
    return x * y

add(1, 2)
multiply(1, 2)